In [1]:
!pip install customtkinter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 kB 4.5 MB/s eta 0:00:00


In [3]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 41.6 MB/s eta 0:00:00


In [8]:
!pip install ipywidgets
from IPython.display import display, clear_output
import ipywidgets as widgets
import cv2
from PIL import Image
import numpy as np
import mediapipe as mp
import pandas as pd
import pickle

# Load landmarks
from landmarks import landmarks

class VideoStream:
    def __init__(self):
        self.video_stream = cv2.VideoCapture(0)
        self.image_widget = widgets.Image(format='jpeg', width=480, height=360)
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
        self.model = self.load_model()

    def load_model(self):
        with open('deadlift.pkl', 'rb') as f:
            model = pickle.load(f)
        return model

    def display(self):
        display(self.image_widget)
        self.update()

    def reset_counter(self):
        self.counter = 0

    def update(self, change=None):
        try:
            ret, frame = self.video_stream.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = self.pose.process(frame_rgb)
                self.mp_drawing.draw_landmarks(frame_rgb, results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                                               self.mp_drawing.DrawingSpec(color=(106, 13, 173), thickness=4, circle_radius=5),
                                               self.mp_drawing.DrawingSpec(color=(255, 102, 0), thickness=5, circle_radius=10))

                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                X = pd.DataFrame([row], columns=landmarks)
                bodylang_prob = self.model.predict_proba(X)[0]
                bodylang_class = self.model.predict(X)[0]

                if bodylang_class == "down" and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                    current_stage = "down"
                elif current_stage == "down" and bodylang_class == "up" and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                    current_stage = "up"
                    self.counter += 1

                img_pil = Image.fromarray(frame_rgb)
                img_byte_array = Image.Image.tobytes(img_pil, format='jpeg')
                self.image_widget.value = img_byte_array

        except Exception as e:
            print(e)

        # Schedule the next update
        self.image_widget.observe(self.update, names='value')

# Create an instance of VideoStream
video_stream = VideoStream()

# Display the video stream
video_stream.display()

Image(value=b'', format='jpeg', height='360', width='480')